# Building blocks for programming preconditioners

In [1]:
import netgen.gui
from netgen.geom2d import unit_square
from ngsolve import *
%gui tk
from ngsolve.la import EigenValues_Preconditioner
mesh = Mesh(unit_square.GenerateMesh(maxh=0.1))
Draw (mesh)

In [2]:
fes = H1(mesh, order=3, dirichlet="left|bottom")
u, v = fes.TnT()
a = BilinearForm(fes)
a += SymbolicBFI(grad(u)*grad(v) + u*v)
a.Assemble()
f = LinearForm(fes)
f += SymbolicLFI(1*v)
f.Assemble()
gfu = GridFunction(fes)

## Create a Jacobi-preconditioner

Let  $A=$ `a.mat` be the assembled matrix, which can be decomposed based on `FreeDofs` ($F$) the remainder ($D$), as in $\S$[1.3](unit-1.3-dirichlet/dirichlet.ipynb),

$$
A = \left( \begin{array}{cc} A_{FF} & A_{FD} \\ A_{DF} & A_{DD} \end{array} \right). 
$$

Then the matrix form of the **point Jacobi preconditioner** is

$$
J = \left( \begin{array}{cc} \text{diag}(A_{FF})^{-1} & 0  \\ 0 & I  \end{array} \right),
$$

which can be obtained in NGSolve using `CreateSmoother`:

In [3]:
preJpoint = a.mat.CreateSmoother(fes.FreeDofs())

NGSolve also gives us a facility to quickly check an estimate of the condition number of the preconditioned matrix by applying the Lanczos algorithm on the preconditioned system.


In [4]:
lams = EigenValues_Preconditioner(mat=a.mat, pre=preJpoint)
lams

 0.0146975
 0.056739
 0.0957295
 0.111941
 0.15266
 0.201398
 0.264173
 0.335984
 0.449808
 0.534789
 0.638433
 0.75197
 0.833963
 0.933586
 1.03869
 1.16685
 1.25086
 1.36964
 1.47475
 1.57987
 1.71008
 1.81181
 1.94538
 2.03796
 2.14448
 2.21284
 2.29168
 2.37038
 2.42404
 2.46758
 2.50565
 2.54039
  2.8097

An estimate of the condition number 
$$
\kappa = \frac{\lambda_{\text{max}} }{ \lambda_{\text{min}}}
$$
is therefore given as follows:

In [5]:
max(lams)/min(lams)

191.16856326877746

One might wonder if we have gained anything by this point Jacobi preconditioning. What if we did not precondition at all? Not preconditioning is the same as preconditioning by the identity operator on $F$-dofs. One way to realize this identity operator in NGSolve is through the projection into the space of free dofs (i.e., the space spanned by the shape functions corresponding to free dofs).

In [6]:
preI = Projector(mask=fes.FreeDofs(), range=True)

In [7]:
lams = EigenValues_Preconditioner(mat=a.mat, pre=preI)
max(lams)/min(lams)

1575.760194898856

Clearly the point Jacobi preconditioner has reduced the condition number.

We can use preconditioners within iterative solvers provided by NGSolve's `solvers` module (which has `MinRes`, `QMR` etc.) Here is an illustration of its use within CG, or the **conjugate gradient** solver: 

In [8]:
solvers.CG(mat=a.mat, pre=preJpoint, rhs=f.vec, sol=gfu.vec)
Draw(gfu)

it =  0  err =  0.05522230048185231
it =  1  err =  0.09382744174195908
it =  2  err =  0.10587866002039001
it =  3  err =  0.09033871153608698
it =  4  err =  0.10086392272140006
it =  5  err =  0.085171193550863
it =  6  err =  0.08906575044224517
it =  7  err =  0.07684062913929465
it =  8  err =  0.05604717506189509
it =  9  err =  0.04472071103380234
it =  10  err =  0.03419741252004595
it =  11  err =  0.02682484049645102
it =  12  err =  0.022322983522404777
it =  13  err =  0.02005619932684475
it =  14  err =  0.01437255908788448
it =  15  err =  0.010772925966275736
it =  16  err =  0.009633149024762151
it =  17  err =  0.007301532183682136
it =  18  err =  0.004811737390491994
it =  19  err =  0.0032534005521665313
it =  20  err =  0.0019025806942343465
it =  21  err =  0.001285747960211658
it =  22  err =  0.0010097069801732897
it =  23  err =  0.0006971063184209079
it =  24  err =  0.0004415360534270245
it =  25  err =  0.00031313626798158364
it =  26  err =  0.000245934548

##  Gauss-Seidel smoothing

The *same* point Jacobi smoother object can also used to perform **point Gauss-Seidel** smoothing. One step of the classical Gauss-Seidel iteration is realized by the method `preJpoint.Smooth()`. Its well known that this iteration converges for matrices like $A$. Below we show how to use it as a linear iterative solver. 


In [9]:
gfu.vec[:] = 0
res = f.vec.CreateVector()              # residual 
projres = f.vec.CreateVector()          # residual projected to freedofs
proj = Projector(fes.FreeDofs(), True)

for i in range(500):
    preJpoint.Smooth(gfu.vec, f.vec)    # one step of point Gauss-Seidel
    res.data = f.vec - a.mat*gfu.vec      
    projres.data = proj * res
    print ("it#", i, ", res =", Norm(projres))
Draw (gfu)

it# 0 , res = 0.08192679624402943
it# 1 , res = 0.07726762555864525
it# 2 , res = 0.07377040495808554
it# 3 , res = 0.07062220895606912
it# 4 , res = 0.06777582421551728
it# 5 , res = 0.06516770753241387
it# 6 , res = 0.06275317789329397
it# 7 , res = 0.060500896902164515
it# 8 , res = 0.05838752669240999
it# 9 , res = 0.056394947347622
it# 10 , res = 0.05450869939951332
it# 11 , res = 0.05271702122673419
it# 12 , res = 0.05101020777328184
it# 13 , res = 0.04938015983450582
it# 14 , res = 0.04782005439140271
it# 15 , res = 0.046324095963871346
it# 16 , res = 0.04488732463071546
it# 17 , res = 0.043505465181981715
it# 18 , res = 0.04217480703984738
it# 19 , res = 0.040892107742331124
it# 20 , res = 0.03965451479597037
it# 21 , res = 0.03845950204060294
it# 22 , res = 0.03730481759400026
it# 23 , res = 0.036188441106383605
it# 24 , res = 0.03510854854333059
it# 25 , res = 0.034063483084375
it# 26 , res = 0.03305173100814238
it# 27 , res = 0.03207190165593137
it# 28 , res = 0.031122710739

it# 270 , res = 2.3760716095100558e-05
it# 271 , res = 2.3066642706964406e-05
it# 272 , res = 2.23928438707072e-05
it# 273 , res = 2.173872734703263e-05
it# 274 , res = 2.1103718196522718e-05
it# 275 , res = 2.0487258274587276e-05
it# 276 , res = 1.988880574025208e-05
it# 277 , res = 1.9307834580570488e-05
it# 278 , res = 1.874383414763832e-05
it# 279 , res = 1.819630871040854e-05
it# 280 , res = 1.7664777018100502e-05
it# 281 , res = 1.7148771878047887e-05
it# 282 , res = 1.6647839744975327e-05
it# 283 , res = 1.616154032154333e-05
it# 284 , res = 1.5689446172416218e-05
it# 285 , res = 1.523114234783743e-05
it# 286 , res = 1.4786226019052763e-05
it# 287 , res = 1.435430612452155e-05
it# 288 , res = 1.3935003025855365e-05
it# 289 , res = 1.352794817427843e-05
it# 290 , res = 1.3132783786722717e-05
it# 291 , res = 1.274916253133673e-05
it# 292 , res = 1.2376747222120524e-05
it# 293 , res = 1.2015210522487429e-05
it# 294 , res = 1.1664234657800392e-05
it# 295 , res = 1.1323511136047182e-

## Implement a forward-backward GS preconditioner

The *same* point Jacobi smoother object is also able to perform a Gauss-Seidel iteration after reversing the ordering of the points, i.e., a **backward** Gauss-Seidel sweep. One can combine the forward and backward sweeps to construct a symmetric preconditioner, often called the **symmetrized Gauss-Seidel preconditioner**. This offers a good illustration of how to construct NGSolve preconditioners from within python. 

In [10]:
class SymmetricGS(BaseMatrix):
    def __init__ (self, smoother):
        super(SymmetricGS, self).__init__()
        self.smoother = smoother
    def Mult (self, x, y):
        y[:] = 0.0
        self.smoother.Smooth(y, x)
        self.smoother.SmoothBack(y,x)
    def Height (self):
        return self.smoother.height
    def Width (self):
        return self.smoother.height

In [11]:
preGS = SymmetricGS(preJpoint)
solvers.CG(mat=a.mat, pre=preGS, rhs=f.vec, sol=gfu.vec)
Draw (gfu)

it =  0  err =  0.0942142968361141
it =  1  err =  0.12242092674801892
it =  2  err =  0.08627911974342342
it =  3  err =  0.054294636877542594
it =  4  err =  0.02683495364136965
it =  5  err =  0.014442946534311297
it =  6  err =  0.006828162088958507
it =  7  err =  0.0029475389328079272
it =  8  err =  0.0009884874866274702
it =  9  err =  0.0003712211220093576
it =  10  err =  0.00011770383684169365
it =  11  err =  4.7294009691925605e-05
it =  12  err =  1.7959564043452263e-05
it =  13  err =  7.73981616121223e-06
it =  14  err =  3.688537792536606e-06
it =  15  err =  1.7853513785616278e-06
it =  16  err =  6.793337817618405e-07
it =  17  err =  3.267271476258278e-07
it =  18  err =  1.0509766947753473e-07
it =  19  err =  5.2632976611271786e-08
it =  20  err =  1.7518834053967536e-08
it =  21  err =  7.174942104447036e-09
it =  22  err =  2.571860623678862e-09
it =  23  err =  7.674623733376341e-10
it =  24  err =  3.231702944081672e-10
it =  25  err =  7.901568725512937e-11
it

In [12]:
lams = EigenValues_Preconditioner(mat=a.mat, pre=preGS)
max(lams)/min(lams)

20.16556918875511

Note that the condition number now is better than that of the system preconditioned by point Jacobi.

## A Block Jacobi preconditioner

The point Jacobi preconditioner is based on inverses of 1 x 1 diagonal blocks.  Condition numbers can be improved by using larger blocks. It is possible to group dofs into blocks within python and construct an NGSolve preconditioner based on the blocks.

Here is an example that constructs vertex-based blocks.

In [13]:
blocks = []
freedofs = fes.FreeDofs()
for v in mesh.vertices:
    vdofs = set()
    for el in mesh[v].elements:
        vdofs |= set(d for d in fes.GetDofNrs(el) if freedofs[d])
    blocks.append (vdofs)
print (blocks)

[{866, 158, 159}, {13, 206, 207, 48, 145, 144, 882, 142, 143, 210, 211, 884}, {256, 257, 2, 901, 146, 147, 148, 21, 22, 149}, {65, 917, 150, 151, 919, 154, 155, 30, 310, 311, 314, 315}, {158, 159, 160, 161, 866, 867, 164, 165, 935, 40, 361, 360}, {160, 161, 164, 165, 166, 167, 40, 41, 170, 171, 867, 868, 869, 362, 363}, {166, 167, 41, 170, 171, 42, 172, 173, 176, 177, 868, 870, 871, 368, 369}, {42, 43, 172, 173, 176, 177, 178, 179, 182, 183, 870, 872, 873, 374, 375}, {43, 44, 178, 179, 182, 183, 184, 185, 188, 189, 872, 874, 875, 380, 381}, {386, 387, 44, 45, 184, 185, 188, 189, 190, 191, 194, 195, 874, 876, 877}, {392, 393, 45, 46, 190, 191, 194, 195, 196, 197, 200, 201, 876, 878, 879}, {398, 399, 46, 47, 196, 197, 200, 201, 202, 203, 204, 205, 878, 880, 881}, {144, 145, 404, 405, 47, 48, 202, 203, 204, 205, 206, 207, 880, 882, 883}, {13, 142, 143, 144, 145, 210, 211, 14, 208, 209, 212, 213, 216, 217, 410, 411, 48, 49, 884, 885, 886}, {13, 14, 15, 208, 209, 212, 213, 214, 215, 216, 21

`CreateBlockSmoother` can now take these blocks and construct a block Jacobi preconditioner.

In [14]:
blockjac = a.mat.CreateBlockSmoother(blocks)

lams = EigenValues_Preconditioner(mat=a.mat, pre=blockjac)
max(lams)/min(lams)

34.81465641222258

Multiplicative smoothers and its symmetrized version often yield better condition numbers in practice. We can apply the same code we wrote above for symmetrization (`SymmetricGS`) to the block smoother:

In [15]:
blockgs = SymmetricGS(blockjac)

lams = EigenValues_Preconditioner(mat=a.mat, pre=blockgs)
max(lams)/min(lams)

2.9825775300409503

## Add a coarse grid correction

Dependence of the condition number on degrees of freedom can often be reduced by preconditioners that appropriately use a coarse grid correction.  It is also possible to experiment with coarse grid corrections using NGSolve's python interface. We now show how to precondition with a coarse grid correction made using the lowest order subspace of `fes`.

In the example below, note that we use `fes.GetDofNrs` again. Previously we used it with argument `el` of type `ElementId`, while now we use it with an argument `v` of type `MeshNode`.

In [16]:
vertexdofs = BitArray(fes.ndof)
vertexdofs[:] = False

for v in mesh.vertices:
    for d in fes.GetDofNrs(v):
        vertexdofs[d] = True
        
vertexdofs &= fes.FreeDofs()

print(vertexdofs)    # bit array, printed 50 chars/line

0: 00100000000001111111111111111110000000001111111111
50: 11111111111111111111111111111111111111111111111111
100: 11111111111111111111111111111111111100000000000000
150: 00000000000000000000000000000000000000000000000000
200: 00000000000000000000000000000000000000000000000000
250: 00000000000000000000000000000000000000000000000000
300: 00000000000000000000000000000000000000000000000000
350: 00000000000000000000000000000000000000000000000000
400: 00000000000000000000000000000000000000000000000000
450: 00000000000000000000000000000000000000000000000000
500: 00000000000000000000000000000000000000000000000000
550: 00000000000000000000000000000000000000000000000000
600: 00000000000000000000000000000000000000000000000000
650: 00000000000000000000000000000000000000000000000000
700: 00000000000000000000000000000000000000000000000000
750: 00000000000000000000000000000000000000000000000000
800: 00000000000000000000000000000000000000000000000000
850: 0000000000000000000000000000000000000000000000

Thus we have made a mask `vertexdofs` which reveals all free dofs associated to vertices. If these are labeled $c$ (and the remainder is labeled $f$), then the matrix $A$ can partitioned into 
$$
A = \left( \begin{array}{cc} A_{cc} & A_{cf} \\ A_{fc} & A_{ff} \end{array} \right). 
$$
The matrix `coarsepre` below represents
$$
\left( \begin{array}{cc} A_{cc}^{-1} & 0 \\ 0 & 0 \end{array} \right). 
$$

In [17]:
coarsepre = a.mat.Inverse(vertexdofs)

This matrix can be used for coarse grid correction. 

*Pitfall!*  Note that `coarsepre` is not appropriate as a preconditioner by itself as it has a large null space. You might get the wrong idea from the results of a Lanczos eigenvalue estimation:

In [18]:
EigenValues_Preconditioner(mat=a.mat, pre=coarsepre)

       1

But this result only gives the Laczos eigenvalue estimates on the *range* of the preconditioner. The preconditioned operator in this case is simply 
$$
\left( \begin{array}{cc} A_{cc}^{-1} & 0 \\ 0 & 0 \end{array} \right)
\left( \begin{array}{cc} A_{cc} & A_{cf} \\ A_{fc} & A_{ff} \end{array} \right)
 = 
 \left( \begin{array}{cc} I  & A_{cc}^{-1} A_{cf} \\ 0 & 0  \end{array} \right),
$$
which is a projection into the $c$-dofs. Hence its no surprise that Lanczos estimated the eigenvalues of this operator (on its range) to be just one. But this does not imply that the condition number of this preconditioned system is nice.

One well-known and correct way to combine the coarse grid correction with one of the previous smoothers is to combine them additively,  to get an **additive two-grid preconditioner** as follows.

In [19]:
twogrid = coarsepre + blockgs 

This addition of two operators (of type `BaseMatrix`) results in another operator, which is stored as an expression, to be evaluated only when needed.  The 2-grid preconditioner has a very good condition number.

In [20]:
EigenValues_Preconditioner(mat=a.mat, pre=twogrid)

 0.992845
 0.997759
 0.99996
 1.34206
 1.80313
 1.83857
 1.96084
 1.98091
 1.99984

Another well-known and correct way is to combine them multiplicatively.